In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import sqlalchemy as sql
import psycopg2
import matplotlib.pyplot as plt
import numpy as np
import os

from dotenv import load_dotenv
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
   
from sqlalchemy import create_engine

# load up the variables from the .env file.
load_dotenv()

True

In [2]:
# File to Load (Remember to Change These)
file_to_load = "Resources/usretechnicalpotential.csv"

# Read Renewable energy potential file and store into Pandas data frame
renewable_energy_data = pd.read_csv(file_to_load)
renewable_energy_data.head()

,Unnamed: 0,urbanUtilityScalePV_GWh,urbanUtilityScalePV_GW,urbanUtilityScalePV_km2,ruralUtilityScalePV_GWh,ruralUtilityScalePV_GW,ruralUtilityScalePV_km2,rooftopPV_GWh,rooftopPV_GW,CSP_GWh,...,biopowerGaseous_GWh,biopowerGaseous_GW,biopowerGaseous_Tonnes-CH4,geothermalHydrothermal_GWh,geothermalHydrothermal_GW,EGSGeothermal_GWh,EGSGeothermal_GW,hydropower_GWh,hydropower_GW,hydropower_countOfSites
0,Alabama,35850,20,426,3706838,2114,44058,15475.0,12,0,...,1533,0,326186,0,0,535489.0,67.0,4102,0,2435
1,Alaska,166,0,2,8282976,9005,187608,NaN,1,0,...,61,0,13156,15437,1,NaN,NaN,23675,5,3053
2,Arizona,121305,52,1096,11867693,5147,107230,22736.0,14,12544333,...,837,0,178188,8329,1,1239147.0,157.0,1303,0,1958
3,Arkansas,28960,15,332,4986388,2747,57239,8484.0,6,0,...,1063,0,226178,0,0,628621.0,79.0,6093,1,3268
4,California,246008,111,2320,8855917,4010,83549,106411.0,75,8490916,...,15510,1,3300211,130921,16,1344179.0,170.0,30023,6,9692


In [ ]:
# set the variables for the pull from SQL.  username and password are in an .env file.  You will need those to make this run
# on your machine.

host = 'localhost'
port = 5432
dbname='ETL_Project'
username = os.environ.get('USER_NAME')
pwd = os.environ.get('PASSWORD')


In [ ]:
# set up the connection to the database "ETL_Project"

conn = psycopg2.connect("host='{}' port={} dbname='{}' user={} password={}".format(host, port, dbname, username, pwd))
sql = "SQL statement"
dat = sqlio.read_sql_query(sql, conn)